#### If you experience slow notebook loading with the Binder service, please try again later, or use a local Jupyter instance.

In [1]:
try:
    import IPython
except:
    !pip install IPython
    import IPython 
from IPython.core.display import HTML
from IPython.display import display, IFrame, HTML

# add stylesheet for notebook
HTML("""<link rel="stylesheet" type="text/css" href="src/css/notebook.css">""")

# Linked Art Visualisation of John Ruskin  


This notebook is concerned with a visualisation of artworks represented in Linked Art abd demonstrates how a timeline visualisation of John Ruskin's artworks can be created using:
- collections data transformed to Linked Art
- a script to transform the Linked Art JSON-LD representation to a JSON input file
- the KnightLab visualisation service

The JSON-LD files for John Ruskin's artworks were created with other notebooks:
- [Ruskin Transformation notebook](01-06-Transform-John-Ruskin.ipynb)
- [Ruskin Reconciliation notebook](02-01-Reconcile-John-Ruskin-Place-Names.ipynb)

KnightLab has a visualisation service that allows you to create an interactive timeline visualisation very quickly and easily. The timeline visualisation can take as input as Google speadsheet or a local CSV file. This notebook described first how to create the visualisation with a Google spreadsheet, and later how to use a local CSV file.

From [http://timeline.knightlab.com/](http://timeline.knightlab.com/):
<blockquote>
TimelineJS is an open-source tool that enables anyone to build visually rich, interactive timelines. Beginners can create a timeline using nothing more than a Google spreadsheet, like the one we used for the Timeline above. Experts can use their JSON skills to create custom installations, while keeping TimelineJS's core functionality.
</blockquote>

The method for creating a visualisation is a simple one:
- use a script to transform Linked Art JSON-LD files to a CSV format as defined by KnightLab
 - [Google spreadsheet template for a timeline](https://docs.google.com/spreadsheets/d/1pHBvXN7nmGkiG8uQSUB82eNlnL8xHu6kydzH_-eguHQ/copy)

The method then differs depending on use of a Google spreadsheet vs local CSV file.

#### Further Reading
- [KnightLab visualisation service](https://knightlab.northwestern.edu/projects/)
- [Google docs](https://doc.google.com)

Detailed requirements and instructions are available at:
- [http://timeline.knightlab.com/](http://timeline.knightlab.com/)


## Read in Linked Art files and transform to CSV timeline template

In [8]:
ruskin_la_files = "./data/ruskin/output/json/"

In [9]:
import os
try:
    import json
except:
    !pip install json
    import json 

import csv

 
sources = {}
sources["https://www.nga.gov/collection/"] = {"name":"National Gallery of Art","colour": "#E8A798"}
sources["https://clevelandart.org/art/"] = {"name":"Cleveland Museum of Art", "colour":"#EDD59E"}
sources["https://www.philamuseum.org/collection/object/"] = {"name":"Philadelphia Museum of Art","colour":"#6B5876"}
sources["https://www.tate.org.uk/art/artworks/"] = {"name":"Tate Museum","colour":"#A09998"}
sources["https://www.harvardartmuseums.org/collections/object/"] = {"name":"Harvard Art Museum","colour":"#00758F"}
sources["ashmolean"] = {"name":"Ashmolean Museum","colour":"#3f83e8"}
sources["rijk"] = {"name":"Rijksmuseum","colour":"#94b7c3"}

  
json_all = {}

# get list of Linked Art JSON-LD files for John Ruskin, created with 01-06-Transform-John-Ruskin.ipynb and 02-01-Reconcile-John-Ruskin-Place-Names.ipynb
file_list=os.listdir(r"./data/ruskin/output/json/")
 
    
for file in file_list:
    with open( ruskin_la_files + file) as json_file:
        json_text = json.load(json_file)
        json_all.update({file : json_text})
        
my_dict = []

title_rows = [
     {
        "Year": "",
        "Month" : "",
        "Day" : "",
        "Time" : "",
        "End Year": "",
        "End Month" : "",
        "End Day" : "",
        "End Time" : "",
        "Display Date": "",
        "Headline" :   "John Ruskin" ,
        "Text": "<p>This timeline visualisation shows artworks created by John Ruskin.</p><p>It demonstrates how the Linked Art data model can be used to transform, reconcile and visualise collections data for artworks.</p> <p>See https://linked.art for more information.</p>",
        "Media": "https://upload.wikimedia.org/wikipedia/commons/0/0a/John_Ruskin_1863.jpg",
        "Media Credit": "Wikipedia",
        "Media Caption": "John Ruskin",
        "Media Thumbnail" : "",
        "Type": "title",
        "Group" :"",
        "Background": ""   },
    
   {
        "Year": "1819",
        "Month" : 2,
        "Day" : 8,
        "Time" : "",
        "End Year": "1900",
        "End Month" : 1,
        "End Day" : 20,
        "End Time" : "",
        "Display Date": "",
        "Headline" :   "John Ruskin's lifetime" ,
        "Text": "",
        "Media": "https://en.wikipedia.org/wiki/John_Ruskin#/media/File:John_Ruskin_1863.jpg",
        "Media Credit": "Wikipedia",
        "Media Caption": "John Ruskin",
        "Media Thumbnail" : "",
        "Type": "era",
        "Group" :"",
        "Background": ""},
    
]

my_dict = title_rows

for file in json_all:
    artwork = json_all[file]
    
    id = artwork["id"]
    
    if "_label" not in artwork:
        continue
   
    if "produced_by" not in artwork:
        continue
    
    if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
        continue
        
    if artwork["produced_by"]["timespan"]["begin_of_the_begin"] == 1819:
        continue
    
    credit = ""
    for source in list(sources.keys()):
            if source in id:
                credit = sources[source]["name"]
                bgcolour = sources[source]["colour"]
            
    text = ""
    imageurl = ""
    homepage = ""
    
    if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
        text = artwork["referred_to_by"][0]["content"]
      
    
    if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
        homepage = artwork["subject_of"][0]["id"]
        text = text + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
   
    if artwork["representation"][0]["id"] != "":
            imageurl = artwork["representation"][0]["id"]
   
    if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
        begin = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
    try:
        begin = int(begin)
    except:
        begin = ""
    
    if "end_of_the_end" in artwork["produced_by"]["timespan"]:
        end = artwork["produced_by"]["timespan"]["end_of_the_end"]
    try:
        end = int(end)
    except:
        end = ""
        
    if abs(begin-end)>10:
        continue
    if imageurl == "" or "placeholder" in imageurl:
        continue

    row = {
        "Year": begin,
        "Month" : 1,
        "Day" : 1,
        "Time" : "",
        "End Year": end,
        "End Month" : 12,
        "End Day" : 31,
        "End Time" : "",
        "Display Date": artwork["produced_by"]["timespan"]["_label"],
        "Headline" :   artwork["_label"] ,
        "Text": text,
        "Media": imageurl,
        "Media Credit": credit,
        "Media Caption": "",
        "Media Thumbnail" :  imageurl,
        "Type": "title",
        "Group" :"",
        "Background": bgcolour     
    }
    my_dict.append(row)

with open('./data/ruskin/output/csv/ruskin_vis.csv', 'w') as f:  
    w = csv.DictWriter(f, my_dict[0].keys())
    w.writeheader()
    for row in my_dict:
        w.writerow(row)
       

## Result - Timeline CSV File

The result of the script transformation is a CSV timeline file, as shown below:

In [10]:
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

#spreadsheet used as input to timeline visualisation
file="data/ruskin/output/csv/ruskin_vis.csv"
mpg = pd.read_csv(file)
mpg.head()

,Year,Month,Day,Time,End Year,End Month,End Day,End Time,Display Date,Headline,Text,Media,Media Credit,Media Caption,Media Thumbnail,Type,Group,Background
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,John Ruskin,<p>This timeline visualisation shows artworks ...,https://upload.wikimedia.org/wikipedia/commons...,Wikipedia,John Ruskin,NaN,title,NaN,NaN
1,1819.0,2.0,8.0,NaN,1900.0,1.0,20.0,NaN,NaN,John Ruskin's lifetime,NaN,https://en.wikipedia.org/wiki/John_Ruskin#/med...,Wikipedia,John Ruskin,NaN,era,NaN,NaN
2,1870.0,1.0,1.0,NaN,1870.0,12.0,31.0,NaN,before 1870,Engraving of Ruskin's Drawing of the Petal Vau...,<br/><br/><a target='_new' href='https://colle...,https://collections.ashmolean.org/media/ashmol...,Ashmolean Museum,NaN,https://collections.ashmolean.org/media/ashmol...,title,NaN,#3f83e8
3,1869.0,1.0,1.0,NaN,1869.0,12.0,31.0,NaN,early November 1869,Enlarged Study of a Prawn's Rostrum,<br/><br/><a target='_new' href='https://colle...,https://collections.ashmolean.org/media/ashmol...,Ashmolean Museum,NaN,https://collections.ashmolean.org/media/ashmol...,title,NaN,#3f83e8
4,1852.0,1.0,1.0,NaN,1852.0,12.0,31.0,NaN,1852(?),Study of a Venetian Capital,"Fine Arts Department, Harvard University, Camb...",https://nrs.harvard.edu/urn-3:HUAM:VRS48802_dynmc,Harvard Art Museum,NaN,https://nrs.harvard.edu/urn-3:HUAM:VRS48802_dynmc,title,NaN,#00758F


## Timeline using Google Spreadsheet

Using a Google spreadsheet, the method continues:
- load the CSV file into a Google Sheet
- Publish the Google spreadsheet to the web
- Copy the URL for the spreadsheet
- Generate timeline by pasting URL into form field at step 3 on the [timeline service page](http://timeline.knightlab.com/#make-step-3)
- Share the timeline

- [Making a timeline from a Google spreadsheet](http://timeline.knightlab.com/docs/using-spreadsheets.html)



### Load CSV File into Google Sheets and Publish to Web

The next step is to 
- load the CSV file into Google sheets
- publish file to web
- copy Google sheet URL

Follow instructions at https://timeline.knightlab.com/#make including 
- paste URL into the relevant form to generate the visualisation
- copy code to share the visualisation

### Result - Timeline

The result is a timeline visualisation

In [11]:
vis = "https://cdn.knightlab.com/libs/timeline3/latest/embed/index.html?source=1mQjqB9EowQTJY_G7nlDoN71In3fZqjuuqu1olq5OQ2g&font=Default&lang=en&timenav_position=top&initial_zoom=2&height=650"

HTML("<a href='" + vis + "' target='_new'>Timeline visualisation</a>")

In [6]:
display(IFrame(vis, width='100%', height=700))

## Timeline Using a Local CSV file

The following HTML code shows how to publish a timeline using a local CSV file

In [12]:
%%HTML
<!-- 1 -->
        <link title="timeline-styles" rel="stylesheet" 
              href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css">

        <!-- 2 -->
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>

        <div id='timeline-embed' style="width: 100%; height: 700px"></div>

        <!-- 3 -->
        <script type="text/javascript">   
            window.timeline = new TL.Timeline('timeline-embed', "data/ruskin/output/vis.json");
        </script>

## Next Steps

A number of videos are available that describe the process of creating hte timeline visualisation

- https://vimeo.com/143407878
- https://www.youtube.com/watch?v=ZUVUjt7jd1c

A [code notebook](03-04-Visualise-John-Ruskin-Story-Map.ipynb) describes how to use the KnightLab StoryMap visualisation service.